---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
def load_data():
    train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test = pd.read_csv('test.csv')
    addresses = pd.read_csv('addresses.csv') 
    latlons = pd.read_csv('latlons.csv')
    return train[~train.compliance.isnull()], test, addresses, latlons

train, test, addresses, latlons = load_data()

c:\python27\lib\site-packages\IPython\core\interactiveshell.py:2822: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [2]:
train.shape

(159880, 34)

In [3]:
test.shape

(61001, 27)

In [4]:
with pd.option_context('display.width', 1000):
    print ('train shape: {}'.format(train.shape))
    print (train.columns)
    print ()
    print ('test shape: {}'.format(test.shape))
    print (test.columns)
    print ()
    print ('addresses shape: {}'.format(addresses.shape))
    print (addresses.columns)
    print ()
    print ('latlons shape: {}'.format(latlons.shape))
    print (latlons.columns)
    print ()

train shape: (159880, 34)
Index([u'ticket_id', u'agency_name', u'inspector_name', u'violator_name', u'violation_street_number', u'violation_street_name', u'violation_zip_code', u'mailing_address_str_number', u'mailing_address_str_name', u'city', u'state', u'zip_code', u'non_us_str_code', u'country', u'ticket_issued_date', u'hearing_date', u'violation_code', u'violation_description', u'disposition', u'fine_amount', u'admin_fee', u'state_fee', u'late_fee', u'discount_amount', u'clean_up_cost', u'judgment_amount', u'payment_amount', u'balance_due', u'payment_date', u'payment_status', u'collection_status', u'grafitti_status', u'compliance_detail', u'compliance'], dtype='object')
()
test shape: (61001, 27)
Index([u'ticket_id', u'agency_name', u'inspector_name', u'violator_name', u'violation_street_number', u'violation_street_name', u'violation_zip_code', u'mailing_address_str_number', u'mailing_address_str_name', u'city', u'state', u'zip_code', u'non_us_str_code', u'country', u'ticket_issue

In [5]:
def join_data():
    train, test, addresses, latlons = load_data()
    add_lat_lon = addresses.merge(latlons, on='address', how='left')
    
    test = test.merge(add_lat_lon, on='ticket_id', how='left')
    train = train.merge(add_lat_lon, on='ticket_id', how='left')
    
    for c in ['lat', 'lon', 'state']:
        test[c].fillna(method='pad', inplace=True)
        train[c].fillna(method='pad', inplace=True)
    return test, train
test, train = join_data()

c:\python27\lib\site-packages\IPython\core\interactiveshell.py:2882: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# count unique values 
lst = [(c, len(test[c].unique()), test[c].dtype) for c in test.columns]
for k, v, t in sorted(lst, key=lambda x: (x[2], x[1])):
    print ('{:<40}{:<10}{:<20}'.format(k, v, str(t)))

ticket_id                               61001     int64               
non_us_str_code                         1         float64             
admin_fee                               1         float64             
state_fee                               1         float64             
discount_amount                         14        float64             
late_fee                                44        float64             
fine_amount                             53        float64             
clean_up_cost                           298       float64             
judgment_amount                         503       float64             
violation_street_number                 13999     float64             
lat                                     31264     float64             
lon                                     32625     float64             
country                                 1         object              
grafitti_status                         2         object              
agency

In [7]:
def to_remove():
    to_remove_all = ['non_us_str_code', 'admin_fee', 'state_fee', 'country', 'inspector_name', 'violation_code',
                     'violation_description', 'violation_street_name', 'violation_zip_code', 'violation_street_number',
                     'zip_code', 'city', 'mailing_address_str_number', 'mailing_address_str_name', 'violator_name',
                     'grafitti_status', 'hearing_date', 'ticket_issued_date', 'address']
    to_remove_train = ['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status',
                       'compliance_detail']
    return to_remove_all, to_remove_train

def to_replace():
    return ['agency_name', 'disposition', 'state']

In [8]:
def filter_and_preprocess_data():
    test, train = join_data()
    to_remove_, to_remove_train = to_remove()
    to_replace_ = to_replace()
    
    train['delay'] = (pd.to_datetime(train.hearing_date) - pd.to_datetime(train.ticket_issued_date)).dt.days
    test['delay'] = (pd.to_datetime(test.hearing_date) - pd.to_datetime(test.ticket_issued_date)).dt.days
    test['delay'].fillna(value=int(train.delay.mean()), inplace=True)
    train['delay'].fillna(value=int(train.delay.mean()), inplace=True)
    
    train.drop(to_remove_+to_remove_train, axis=1, inplace=True)
    test.drop(to_remove_, axis=1, inplace=True)
    
    train = pd.get_dummies(train, columns=to_replace_, dummy_na=True)
    test = pd.get_dummies(test, columns=to_replace_, dummy_na=True)
    
    train.drop(set(train.columns) - set(test.columns) - set(['compliance']), axis=1, inplace=True)
    test.drop(set(test.columns) - set(train.columns) , axis=1, inplace=True)
    
    return train.set_index('ticket_id'), test.set_index('ticket_id')
train, test = filter_and_preprocess_data()

c:\python27\lib\site-packages\ipykernel_launcher.py:21: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.


In [9]:
test.shape

(61001, 76)

In [10]:
train.shape

(159880, 77)

In [11]:
from sklearn.preprocessing import MinMaxScaler
def minmax():
    train, test = filter_and_preprocess_data()
    X_train, y_train = train[train.columns[train.columns != 'compliance']], train['compliance']
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(test)
    return X_train_scaled, y_train, X_test_scaled


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

In [13]:
def test_classifier(clf, data):
    X_train, y_train = data
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)
    
    clf.fit(X_train, y_train)
    
    fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
    print('auc {}'.format(auc(fpr, tpr)))
    print('score {}'.format(clf.score(X_test, y_test)))

In [14]:
data = train[train.columns[train.columns != 'compliance']], train['compliance']
data_scaled = minmax()[:2]

c:\python27\lib\site-packages\ipykernel_launcher.py:3: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
from sklearn.linear_model import LogisticRegression

def linear_model():
    return LogisticRegression(penalty='l2', C=1, random_state=0, solver='liblinear') #lbfgs
print ('normal data')
test_classifier(linear_model(), data)
print ('scaled data')
test_classifier(linear_model(), data_scaled)

normal data
auc 0.790885329063
score 0.934676007005
scaled data
auc 0.78259009436
score 0.932349261946


In [16]:
from sklearn.tree import DecisionTreeClassifier

def tree_model():
    return DecisionTreeClassifier(max_depth=8, random_state=0)
print ('normal data')
test_classifier(tree_model(), data)
print ('scaled data')
test_classifier(tree_model(), data_scaled)

normal data
auc 0.802191304944
score 0.942932199149
scaled data
auc 0.802194918685
score 0.942932199149


In [17]:
from sklearn.ensemble import RandomForestClassifier

def ensemble_model():
    return RandomForestClassifier(n_estimators=100, max_depth=30, random_state=0)
print ('normal data')
test_classifier(ensemble_model(), data)
print ('scaled data')
test_classifier(ensemble_model(), data_scaled)

normal data
auc 0.832978449901
score 0.945183887916
scaled data
auc 0.83324452388
score 0.9453340005


In [18]:
from sklearn.ensemble import AdaBoostClassifier

def ensemble_model_2():
    return AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=0)
print ('normal data')
test_classifier(ensemble_model_2(), data)
print ('scaled data')
test_classifier(ensemble_model_2(), data_scaled)

normal data
auc 0.797288217385
score 0.941856392294
scaled data
auc 0.797288217385
score 0.941856392294


In [19]:
from sklearn.neural_network import MLPClassifier

def mlpc_model():
    return MLPClassifier(hidden_layer_sizes=[100, 10], alpha=5, solver='lbfgs', random_state=0)
print ('normal data')
test_classifier(mlpc_model(), data)
print ('scaled data')
test_classifier(mlpc_model(), data_scaled)

normal data
auc 0.767778042879
score 0.933124843633
scaled data
auc 0.794874097982
score 0.939204403302


In [20]:
import pandas as pd
import numpy as np

def blight_model():
    
    clf = ensemble_model()
    
    train, X_test = filter_and_preprocess_data()
    X_train, y_train = train[train.columns[train.columns != 'compliance']], train['compliance']
    
    clf.fit(X_train, y_train)
    
    return pd.Series(clf.predict_proba(X_test)[:, 1], index=X_test.index, name='compliance', dtype=np.float32)
blight_model()

c:\python27\lib\site-packages\ipykernel_launcher.py:8: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  


ticket_id
284932    0.028482
285362    0.067720
285361    0.060761
285338    0.089065
285346    0.077346
285345    0.038454
285347    0.048563
285342    0.926475
285530    0.021784
284989    0.061346
285344    0.038295
285343    0.002093
285340    0.002093
285341    0.107073
285349    0.081086
285348    0.016943
284991    0.019684
285532    0.015108
285406    0.017968
285001    0.002768
285006    0.000194
285405    0.004158
285337    0.004751
285496    0.064849
285497    0.089930
285378    0.002758
285589    0.019920
285585    0.035300
285501    0.095492
285581    0.005842
            ...   
376367    0.020652
376366    0.025522
376362    0.075935
376363    0.218435
376365    0.020652
376364    0.025522
376228    0.161288
376265    0.041875
376286    0.029437
376320    0.030997
376314    0.024506
376327    0.358495
376385    0.278201
376435    0.573333
376370    0.933490
376434    0.037543
376459    0.030049
376478    0.000076
376473    0.036153
376484    0.002414
376482    0.069685
37